# Annexe B - Calcul des anomalies SAR Sentinel-1 (Taal) 
**Objectif :** calculer, pour chaque polarisation, l'anomalie post‑éruption (période fixe après le 12/01/2020) par rapport à la moyenne
des composites de référence (2017–2019), en utilisant un composite *minimum* sur la dimension temporelle.

In [ ]:
## Téléchargement des librairies et modules Python nécessaires 
import rioxarray
import geopandas as gpd
import xarray as xr
import glob
import datetime
import os

# Paramètres 
roi = gpd.read_file("/export/students/cvanvooren/SAR Processing/QGIS/extent_entier.shp") # Limites de la zone d'étude 
polarizations = ["VV","VH"] 
methods = ["min"] # Méthode statistique appliquée 
base_path = "/export/images/Sentinel1/sites/taal/GEE/acquisitions_UTM51N/" # Chemin vers les images SAR 
output_base = "/export/students/cvanvooren/SAR Processing/Anomalies_result/" # Chemin vers le dossier résultat 
eruption_date = datetime.datetime(2020, 1, 12)
durations = {"3mois": 90} # Durée d'analyse de l'anomalie 

# Extraction la date à partir du nom de fichier 
def extract_date(filepath):
    format = '%Y%m%d'
    begin_date = 17
    date_str = os.path.basename(filepath)[begin_date:begin_date+8]
    return datetime.datetime.strptime(date_str, format)

# Fonction création du composite
def create_composite(image_list, method):
    stack = xr.concat(image_list, dim="time")
    if method == "min":
        return stack.min(dim="time")

# Fonction filtre et chargement des images 
def load_images_in_period(all_images, start, end):
    selected = [f for f in all_images if start <= extract_date(f) <= end]
    img_list = []
    for img_path in selected:
        img = rioxarray.open_rasterio(img_path)
        img_cropped = img.rio.clip(roi.geometry.values, roi.crs, drop=True)
        img_cropped = img_cropped.assign_coords(time=extract_date(img_path))
        img_list.append(img_cropped)
    return img_list

# Anomalies
for pol in polarizations:
    all_images = sorted([f for f in glob.glob(os.path.join(base_path, "*.tif")) if pol in f])

    for duration_name, duration_days in durations.items():
        post_start = eruption_date
        post_end = eruption_date + datetime.timedelta(days=duration_days)

        for method in methods:
            # Composite post-éruption 
            post_images = load_images_in_period(all_images, post_start, post_end)
            if not post_images:
                print(f"Pas d'images disponible")
                continue
            post_composite = create_composite(post_images, method)

            # Composites de référence (2017, 2018, 2019) 
            ref_composites = []
            for ref_year in [2017, 2018, 2019]:
                ref_start = post_start.replace(year=ref_year)
                ref_end = post_end.replace(year=ref_year)
                ref_images = load_images_in_period(all_images, ref_start, ref_end)
                if not ref_images:
                    continue
                comp = create_composite(ref_images, method)
                ref_composites.append(comp)

            if not ref_composites:
                print(f"non trouvé")
                continue

            # Moyenne des composites de référence 
            ref_stack = xr.concat(ref_composites, dim="ref")
            ref_mean = ref_stack.mean(dim="ref")

            # Calcul de l'anomalie 
            anomaly = post_composite - ref_mean

            # Sauvegarde 
            os.makedirs(output_base, exist_ok=True)
            out_path = os.path.join(output_base, f"anomaly_extent_entier_{pol}_{duration_name}_{method}.tif")
            anomaly.rio.to_raster(out_path)
